In [94]:
#读入数据
content = []
with open('./wikilens-ratings/out.wikilens-ratings') as fp:
    content = fp.readlines()
    content.remove(content[0])

content

['1\t1\t4\t1165070617\n',
 '1\t2\t5\t1116504904\n',
 '1\t3\t5\t1085579179\n',
 '1\t4\t5\t1081863704\n',
 '1\t5\t3.5\t1129498553\n',
 '1\t6\t5\t1081863704\n',
 '1\t7\t5\t1112186499\n',
 '1\t8\t5\t1112186520\n',
 '1\t9\t3\t1081863704\n',
 '1\t10\t5\t1081863704\n',
 '1\t11\t5\t1081863704\n',
 '1\t12\t5\t1081863704\n',
 '1\t13\t5\t1081863704\n',
 '1\t14\t5\t1081863704\n',
 '1\t15\t3.5\t1081863704\n',
 '1\t16\t5\t1112186495\n',
 '1\t17\t5\t1081863704\n',
 '1\t18\t4.5\t1081863704\n',
 '1\t19\t5\t1081863704\n',
 '1\t20\t3\t1081863704\n',
 '1\t21\t3.5\t1081863704\n',
 '1\t22\t5\t1112186512\n',
 '1\t23\t5\t1112186487\n',
 '1\t24\t5\t1112186458\n',
 '1\t25\t5\t1112186505\n',
 '1\t26\t3.5\t1081863704\n',
 '1\t27\t3.5\t1081863704\n',
 '1\t28\t5\t1081863704\n',
 '1\t29\t3\t1081863704\n',
 '1\t30\t4.5\t1083091842\n',
 '1\t31\t2\t1081863704\n',
 '1\t32\t2\t1160031524\n',
 '1\t33\t5\t1112186510\n',
 '1\t34\t3\t1081863704\n',
 '1\t35\t1\t1081863704\n',
 '1\t36\t5\t1097773999\n',
 '1\t37\t4\t1081863704\

In [96]:
#划分训练集，测试集
import pandas as pd
from sklearn.model_selection import train_test_split

rnames = ['user_id', 'product_id', 'rating', 'timestamp']
data = pd.read_csv('./wikilens-ratings/out.wikilens-ratings',sep = '\t', names=rnames)
data = data.drop(0)

trn_data, val_data, _, _ = train_test_split(data, data, test_size=0.2)

trn_data = trn_data.sort_values(by='user_id',ascending=True)
trn_data.to_csv('trn_data.txt',sep='\t', index=False,header=False)

val_data = val_data.sort_values(by='user_id',ascending=True)
val_data.to_csv('val_data.txt',sep='\t', index=False,header=False)

data

,user_id,product_id,rating,timestamp
1,1,1.0,4.0,1.165071e+09
2,1,2.0,5.0,1.116505e+09
3,1,3.0,5.0,1.085579e+09
4,1,4.0,5.0,1.081864e+09
5,1,5.0,3.5,1.129499e+09
...,...,...,...,...
26933,326,4440.0,0.5,1.201838e+09
26934,326,2685.0,2.0,1.201838e+09
26935,326,2689.0,4.0,1.201838e+09
26936,326,5111.0,4.0,1.201838e+09


In [97]:
#格式化数据集
content_trn = []
content_val = []

with open('./trn_data.txt') as fp:
    content_trn = fp.readlines()

trn_data_dict = {}
for line in content_trn:
    line = line.strip().replace('\n','').split('\t')
    if not line[0] in trn_data_dict.keys():
        trn_data_dict[line[0]] = {line[1]:line[2]}
    else:
        trn_data_dict[line[0]][line[1]] = line[2]

with open('./val_data.txt') as fp:
    content_val = fp.readlines()

val_data_dict = {}
for line in content_val:
    line = line.strip().replace('\n','').split('\t')
    if not line[0] in val_data_dict.keys():
        val_data_dict[line[0]] = {line[1]:line[2]}
    else:
        val_data_dict[line[0]][line[1]] = line[2]


In [113]:
#用皮尔逊相关系数计算测试集中的用户相似度
from math import *

def Euclid(user1,user2):
    #取出两个用户购买的商品
    user1_data = trn_data_dict[user1]
    user2_data = trn_data_dict[user2]

    #储存两个用户都购买过的商品id
    item_set = []

    global user1_item_value
    user1_item_value = 0
    user2_item_value = 0

    #遍历寻找两个用户都购买过的商品
    for key in user1_data.keys():
        if key in user2_data.keys():
            item_set.append(key)
            user1_item_value += float(user1_data[key])
            user2_item_value += float(user2_data[key])
    
    #若两个用户都购买过的商品数为0，则皮尔逊相关系数为0
    if len(item_set) == 0:
        pearson_cc = 0

        return pearson_cc
    #计算用户对共有商品评价的均值
    user1_item_mean = user1_item_value/len(item_set)
    user2_item_mean = user2_item_value/len(item_set)

    #计算皮尔逊系数
    m = 0
    n,n1,n2 = 0,0,0
    for i in item_set:
        m += (float(user1_data[i])-user1_item_mean)*(float(user2_data[i])-user2_item_mean)
        n1 += (float(user1_data[i])-user1_item_mean)**2
        n2 += (float(user2_data[i])-user2_item_mean)**2 
    n = sqrt(n1)*sqrt(n2)
    if n == 0:
        #特殊情况下的pearson_cc计算方式
        pearson_cc = (1-abs(n1-n2))/float(max(len(user1_data.keys()),len(user2_data.keys())))
    else:
        pearson_cc = m/n

    return pearson_cc


In [99]:
#计算某个用户与其他用户的相似度比对
def top_similar(user):
    
    res = []
    
    for userid in trn_data_dict.keys():
        #排除当前用户
        if not userid == user:
            similar = Euclid(user,userid)
            res.append((userid,similar))
    
    res.sort(key = lambda val:val[1],reverse=True)

    return res

In [117]:
top_similar('1')

[('316', 1.0000000000000002),
 ('243', 1.0),
 ('208', 0.9999999999999998),
 ('214', 0.9999999999999998),
 ('216', 0.9999999999999998),
 ('253', 0.9999999999999998),
 ('277', 0.9999999999999998),
 ('7', 0.9999999999999998),
 ('318', 0.9961164901835046),
 ('83', 0.9891004919611718),
 ('205', 0.9707253433941511),
 ('252', 0.917662935482247),
 ('77', 0.9156382231357673),
 ('201', 0.8827348295047496),
 ('48', 0.8735617942666208),
 ('271', 0.8703882797784892),
 ('114', 0.8660254037844385),
 ('63', 0.8660254037844385),
 ('65', 0.8247621221537367),
 ('206', 0.7898810168000062),
 ('90', 0.7610423906471904),
 ('67', 0.7607257743127308),
 ('54', 0.7559289460184544),
 ('81', 0.7396929051097703),
 ('104', 0.7385489458759965),
 ('16', 0.7048983435861615),
 ('221', 0.697373019087618),
 ('308', 0.6866171722688429),
 ('82', 0.6826677292891367),
 ('45', 0.6762430438109829),
 ('268', 0.674199862463242),
 ('123', 0.6720400782134934),
 ('53', 0.6710890224298677),
 ('142', 0.6565596501177023),
 ('42', 0.655

In [103]:
#构建推荐方法
def recommend(user):
    #取相关系数大于0的用户,储存其商品信息
    top_user = top_similar(user)
    items = {}
    for i in range(len(top_user)):
        if top_user[i][1] == 0 or top_user[i][1] < 0:
            top_user = top_user[:i]
            break
        else:
            u = top_user[i][0]
            #记录相似用户购买的且user未购买的评分大于等于3的商品，将这些商品推荐给用户
            for j in range(len(trn_data_dict[u].keys())):
                if float(trn_data_dict[u][list(trn_data_dict[u].keys())[j]]) >= 3 and list(trn_data_dict[u].keys())[j] not in trn_data_dict[user].keys():
                        if not u in items.keys():
                            items[u] = {list(trn_data_dict[u].keys())[j]:trn_data_dict[u][list(trn_data_dict[u].keys())[j]]}
                        else:
                            items[u][list(trn_data_dict[u].keys())[j]] = trn_data_dict[u][list(trn_data_dict[u].keys())[j]]
    
    dict_top_user = dict(top_user)

    #基于用户的协同过滤子引擎，计算用户对推荐商品的喜好程度
    m = {}
    n = {}
    score = {}
    for i in items.keys():
        for j in items[i].keys():
            if not j in m.keys():
                m[j] = (float(items[i][j])*dict_top_user[i])
                n[j] = (dict_top_user[i])
            else:
                m[j] += (float(items[i][j])*dict_top_user[i])
                n[j] += (dict_top_user[i])
    
    for i in m.keys():
        score[i] = m[i]/n[i]
    
    score = sorted(score.items(), key=lambda x:x[1], reverse=True)
    score = dict(score)
    return score

In [115]:
user = '1'
recommend(user)

{'4707.0': 5.0,
 '1008.0': 5.0,
 '4706.0': 5.0,
 '907.0': 5.0,
 '1138.0': 5.0,
 '1149.0': 5.0,
 '1148.0': 5.0,
 '3330.0': 5.0,
 '954.0': 5.0,
 '4652.0': 5.0,
 '4606.0': 5.0,
 '4654.0': 5.0,
 '4651.0': 5.0,
 '2745.0': 5.0,
 '4145.0': 5.0,
 '4160.0': 5.0,
 '4158.0': 5.0,
 '4251.0': 5.0,
 '508.0': 5.0,
 '4265.0': 5.0,
 '1574.0': 5.0,
 '4738.0': 5.0,
 '4737.0': 5.0,
 '936.0': 5.0,
 '912.0': 5.0,
 '2671.0': 5.0,
 '2700.0': 5.0,
 '2725.0': 5.0,
 '2906.0': 5.0,
 '4443.0': 5.0,
 '10.0': 5.0,
 '815.0': 5.0,
 '805.0': 5.0,
 '808.0': 5.0,
 '807.0': 5.0,
 '809.0': 5.0,
 '84.0': 5.0,
 '804.0': 5.0,
 '6.0': 5.0,
 '812.0': 5.0,
 '79.0': 5.0,
 '814.0': 5.0,
 '828.0': 5.0,
 '817.0': 5.0,
 '813.0': 5.0,
 '827.0': 5.0,
 '811.0': 5.0,
 '826.0': 5.0,
 '816.0': 5.0,
 '825.0': 5.0,
 '829.0': 5.0,
 '3934.0': 5.0,
 '1348.0': 5.0,
 '1404.0': 5.0,
 '1052.0': 5.0,
 '1080.0': 5.0,
 '1073.0': 5.0,
 '1078.0': 5.0,
 '1082.0': 5.0,
 '1045.0': 5.0,
 '1050.0': 5.0,
 '988.0': 5.0,
 '1065.0': 5.0,
 '1826.0': 5.0,
 '1785.0

In [116]:
#推荐结果评估
m = 0
n = len(val_data_dict[user].keys())
for i in val_data_dict[user].keys():
    if i in recommend(user).keys():
        m += (1-abs(float(val_data_dict[user][i])-recommend(user)[i])/user1_item_value)

print(m/n)

0.7588378451234132
